In [ ]:
try:
    raw_df = pd.read_csv('../heart-failure-prediction/heart.csv')
except:
    raw_df = pd.read_csv('heart.csv')



In [4]:
df = raw_df

In [5]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [ ]:

from scipy.stats import zscore
numeric = df.select_dtypes(include=['number']).columns
zscore = df[numeric].apply(zscore)
no_out = df[(zscore < 3).all(axis=1) & (zscore > -3).all(axis=1)]
df = no_out
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:

cat = []
cat_col = df.select_dtypes(include=['object']).columns
for col in cat_col:
  cat.append(col)
cat

['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [9]:
for i in cat:
    print(f"{i} --> {df[i].unique()}")

Sex --> ['M' 'F']
ChestPainType --> ['ATA' 'NAP' 'ASY' 'TA']
RestingECG --> ['Normal' 'ST' 'LVH']
ExerciseAngina --> ['N' 'Y']
ST_Slope --> ['Up' 'Flat' 'Down']


In [10]:
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder

ordinal = []
nominal = ['Sex','ExerciseAngina']
for i in cat:
    if i not in nominal:
        ordinal.append(i)
cat_data = {'ord':ordinal,'nom':nominal}
for i in cat_data:
    if i=='ord':
        x = cat_data['ord']
        for j in x:
            encoder = OrdinalEncoder(categories=[df[j].unique()])
            df[j] = encoder.fit_transform(df[[j]])
            
    else:
        x = cat_data['nom']
        for j in x:
            encoder = LabelEncoder()
            df[j] = encoder.fit_transform(df[j])
df.head()

/tmp/ipykernel_17/1284057579.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j] = encoder.fit_transform(df[[j]])
/tmp/ipykernel_17/1284057579.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j] = encoder.fit_transform(df[[j]])
/tmp/ipykernel_17/1284057579.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,0.0,140,289,0,0.0,172,0,0.0,0.0,0
1,49,0,1.0,160,180,0,0.0,156,0,1.0,1.0,1
2,37,1,0.0,130,283,0,1.0,98,0,0.0,0.0,0
3,48,0,2.0,138,214,0,0.0,108,1,1.5,1.0,1
4,54,1,1.0,150,195,0,0.0,122,0,0.0,0.0,0


In [11]:
arr = df.values
arr

array([[40. ,  1. ,  0. , ...,  0. ,  0. ,  0. ],
       [49. ,  0. ,  1. , ...,  1. ,  1. ,  1. ],
       [37. ,  1. ,  0. , ...,  0. ,  0. ,  0. ],
       ...,
       [57. ,  1. ,  2. , ...,  1.2,  1. ,  1. ],
       [57. ,  0. ,  0. , ...,  0. ,  1. ,  1. ],
       [38. ,  1. ,  1. , ...,  0. ,  0. ,  0. ]])

In [12]:
x = arr[:,:-1].astype(int)
y = arr[:,-1].astype(int)
x,y

(array([[40,  1,  0, ...,  0,  0,  0],
        [49,  0,  1, ...,  0,  1,  1],
        [37,  1,  0, ...,  0,  0,  0],
        ...,
        [57,  1,  2, ...,  1,  1,  1],
        [57,  0,  0, ...,  0,  0,  1],
        [38,  1,  1, ...,  0,  0,  0]]),
 array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
        0, 

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)
x

array([[-1.42815446,  0.515943  , -1.69640215, ..., -0.8229452 ,
        -0.74528676, -1.04249607],
       [-0.47585532, -1.93819859, -0.52417653, ..., -0.8229452 ,
         0.33017519,  0.62216462],
       [-1.7455875 ,  0.515943  , -1.69640215, ..., -0.8229452 ,
        -0.74528676, -1.04249607],
       ...,
       [ 0.3706328 ,  0.515943  ,  0.64804909, ...,  1.21514774,
         0.33017519,  0.62216462],
       [ 0.3706328 , -1.93819859, -1.69640215, ..., -0.8229452 ,
        -0.74528676,  0.62216462],
       [-1.63977649,  0.515943  , -0.52417653, ..., -0.8229452 ,
        -0.74528676, -1.04249607]])

In [14]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=50)

In [15]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

models = {"SVM":SVC(),"LogisticRegression":LogisticRegression(),"RandomForest":RandomForestClassifier()}

In [ ]:
# accuracies without pca
from sklearn.metrics import accuracy_score

accuracy = {}

for model_name , model in models.items():
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    acc = accuracy_score(y_test,y_pred)
    accuracy[model_name] = acc
    print(f"{model_name} --> {acc}")

print("Best Model :",max(accuracy,key=accuracy.get))

SVM --> 0.8407407407407408
LogisticRegression --> 0.837037037037037
RandomForest --> 0.8555555555555555
Best Model : RandomForest


In [ ]:
# accuracies without pca
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

pca = PCA(5)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.fit_transform(x_test)
accuracy_pca = {}

for model_name , model in models.items():
    model.fit(x_train_pca,y_train)
    y_pred = model.predict(x_test_pca)
    acc = accuracy_score(y_test,y_pred)
    accuracy_pca[model_name] = acc
    print(f"{model_name} --> {acc}")

print("Best Model :",max(accuracy_pca,key=accuracy_pca.get))

SVM --> 0.2222222222222222
LogisticRegression --> 0.16296296296296298
RandomForest --> 0.1925925925925926
Best Model : SVM
